# Load

In [ ]:
!gdown --fuzzy https://docs.google.com/spreadsheets/d/1f2zvYLYf033tYx3OLmdWaGXsRqk4apUZ/edit?usp=sharing&ouid=100299165652406824721&rtpof=true&sd=true

In [ ]:
!pip install -q openpyxl

In [ ]:
import pandas as pd

file_path = 'maximum_flow_data.xlsx'

try:
  node = pd.read_excel(file_path, engine='openpyxl', sheet_name="Node")
  edge = pd.read_excel(file_path, engine='openpyxl', sheet_name="Edge")
  road = pd.read_excel(file_path, engine='openpyxl', sheet_name="Road")
except FileNotFoundError:
  print(f"Error: File '{file_path}' not found.")
except Exception as e:
  print(f"An error occurred: {e}")

# Nodes

In [ ]:
df_node = node.dropna(subset=['Node','Location'])[['Node','Location']]
df_node[['latitude', 'longitude']] = df_node['Location'].str.split(',', expand=True)
df_node['latitude'] = df_node['latitude'].astype(float)
df_node['longitude'] = df_node['longitude'].astype(float)
df_node.rename(columns={'Node': 'name'}, inplace=True)
df_node['name'] = df_node['name'].str.strip()
df_node = df_node.drop(columns=['Location'])
df_node = df_node.sort_values(by='name')
df_node.reset_index(drop=True, inplace=True)
df_node['id'] = range(0, len(df_node))
df_node.to_csv('nodes.csv', encoding='utf-8', index=False)

# Edges

In [ ]:
df_edge = edge.dropna()
df_edge = df_edge.sort_values(by=['Path'])
df_edge.rename(columns={'From': 'start', 'To': 'end', 'Capacity': 'capacity', 'Street name': 'name', 'Distance (Cost)': 'cost'}, inplace=True)
df_edge['start'] = df_edge['start'].str.strip()
df_edge['end'] = df_edge['end'].str.strip()
df_edge['name'] = df_edge['name'].str.strip()
df_edge['capacity'] = df_edge['capacity'].astype(int)
df_edge['cost'] = df_edge['cost'].astype(int)

node_id_map = dict(zip(df_node['name'], df_node['id']))
node_coordinate_map = dict(zip(df_node['id'], zip( df_node['latitude'].astype(float) ,df_node['longitude'].astype(float))))

df_edge['start_name'] = df_edge['start']
df_edge['end_name'] = df_edge['end']
df_edge['start'] = df_edge['start'].map(node_id_map)
df_edge['end'] = df_edge['end'].map(node_id_map)
df_edge['pair'] = list(zip(df_edge['start'].astype(int), df_edge['end'].astype(int)))

df_edge['start_coordinate'] = df_edge['start'].map(node_coordinate_map)
df_edge['end_coordinate'] = df_edge['end'].map(node_coordinate_map)

df_edge['start_coordinate_str'] = df_edge['start_coordinate'].apply(lambda x: f"{x[0]:.7f}, {x[1]:.7f}".strip())
df_edge['end_coordinate_str'] = df_edge['end_coordinate'].apply(lambda x: f"{x[0]:.7f}, {x[1]:.7f}".strip())
df_edge['node_coordinate'] = df_edge.apply(lambda row: [(row['start_coordinate'][0], row['start_coordinate'][1]), (row['end_coordinate'][0], row['end_coordinate'][1])], axis=1)

df_edge = df_edge.drop(columns=['Path','start_coordinate','end_coordinate'])
df_edge.to_csv('edges.csv', encoding='utf-8', index=False)

# Adjacent Matrix

In [ ]:
import numpy as np

num_nodes = len(df_node)
capacity_adj_matrix = np.zeros((num_nodes, num_nodes), dtype=int)
cost_adj_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

for _, row in df_edge.iterrows():
    start_node = row['start']
    end_node = row['end']
    capacity = row['capacity']
    capacity_adj_matrix[start_node, end_node] = capacity

for _, row in df_edge.iterrows():
    start_node = row['start']
    end_node = row['end']
    cost = row['cost']
    cost_adj_matrix[start_node, end_node] = cost

df_node_dict = df_node.set_index('id').to_dict('index')
df_edge_dict = df_edge.set_index('pair').to_dict('index')

# Export

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1d3qRdIVbxKb_huF-M8rVmcPPZ0f_iDxq/view?usp=sharing

In [ ]:
import pickle

data_to_save = {
    'df_node': df_node,
    'df_edge': df_edge,
    'capacity_adj_matrix': capacity_adj_matrix,
    'cost_adj_matrix': cost_adj_matrix,
    'df_node_dict': df_node_dict,
    'df_edge_dict': df_edge_dict
}

with open('maximum_flow_data.pickle', 'wb') as f:
    pickle.dump(data_to_save, f)

with open('maximum_flow_data.pickle', 'rb') as f:
    loaded_data = pickle.load(f)

df_node = loaded_data['df_node']
df_edge = loaded_data['df_edge']
capacity_adj_matrix = loaded_data['capacity_adj_matrix']
cost_adj_matrix = loaded_data['cost_adj_matrix']
df_node_dict = loaded_data['df_node_dict']
df_edge_dict = loaded_data['df_edge_dict']